# Welcome to Colab Enterprise <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">

Connect to a Runtime and begin!

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from torchvision.models import resnet18
from sklearn.metrics import accuracy_score
import itertools

data_dir = '/content/Dataset 1/Colorectal Cancer '

classes = ['MUS', 'NORM', 'STR']


images = []
labels = []


for class_name in classes:
    class_path = os.path.join(data_dir, class_name)
    for filename in os.listdir(class_path):
        if filename.endswith('.tif'):
            image_path = os.path.join(class_path, filename)
            image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
            images.append(image)
            labels.append(class_name)


X = np.array(images)
y = np.array(labels)


label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

data_transform = transforms.Compose([
    transforms.ToPILImage(),  # Ensure input is a PIL image
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


# Define the hyperparameter search space
hyperparameters = {
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64],
    'epochs': [5, 10, 15]
}

best_val_accuracy = 0.0
best_hyperparameters = {}

# Iterate over all combinations of hyperparameters
for lr, bs, ep in itertools.product(hyperparameters['learning_rate'], hyperparameters['batch_size'], hyperparameters['epochs']):
    print(f"\nTraining with learning rate={lr}, batch size={bs}, epochs={ep}")

    # Convert data to PyTorch tensors for each iteration
    X_train_transformed = torch.stack([data_transform(x) for x in X_train])
    X_val_transformed = torch.stack([data_transform(x) for x in X_val])

    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)

    train_dataset = TensorDataset(X_train_transformed, y_train_tensor)
    val_dataset = TensorDataset(X_val_transformed, y_val_tensor)
    train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=bs, shuffle=False)

    # Create a new model for each iteration
    model = resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, len(classes))
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(ep):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation
        model.eval()
        val_predictions = []
        val_true_labels = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                val_predictions.extend(predicted.numpy())
                val_true_labels.extend(labels.numpy())

        val_accuracy = accuracy_score(val_true_labels, val_predictions)
        print(f"Epoch {epoch + 1}/{ep}, Loss: {running_loss / len(train_loader)}, Validation Accuracy: {val_accuracy}")

    # Update the best hyperparameters if needed
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_hyperparameters = {'learning_rate': lr, 'batch_size': bs, 'epochs': ep}

# Print the best hyperparameters and validation accuracy
print("\nBest Hyperparameters:", best_hyperparameters)
print("Best Validation Accuracy:", best_val_accuracy)

# Save the model with the best hyperparameters
best_model = resnet18(weights=None)
best_model.fc = nn.Linear(best_model.fc.in_features, len(classes))
best_optimizer = optim.Adam(best_model.parameters(), lr=best_hyperparameters['learning_rate'])
best_model.load_state_dict(torch.load('/content/trained_resnet18.pth'))
best_model.eval()

torch.save(best_model.state_dict(), '/content/best_trained_resnet18.pth')



Training with learning rate=0.001, batch size=16, epochs=5
Epoch 1/5, Loss: 0.563397299721837, Validation Accuracy: 0.6333333333333333
Epoch 2/5, Loss: 0.4080287310977777, Validation Accuracy: 0.605
Epoch 3/5, Loss: 0.30784684627006453, Validation Accuracy: 0.85
Epoch 4/5, Loss: 0.2983188282574217, Validation Accuracy: 0.6883333333333334
Epoch 5/5, Loss: 0.2462856774404645, Validation Accuracy: 0.835

Training with learning rate=0.001, batch size=16, epochs=10
Epoch 1/10, Loss: 0.5862876942257086, Validation Accuracy: 0.57
Epoch 2/10, Loss: 0.42840079627931116, Validation Accuracy: 0.7066666666666667
Epoch 3/10, Loss: 0.338975270713369, Validation Accuracy: 0.6033333333333334
Epoch 4/10, Loss: 0.28906801430508494, Validation Accuracy: 0.6783333333333333
Epoch 5/10, Loss: 0.2685334566918512, Validation Accuracy: 0.8933333333333333
Epoch 6/10, Loss: 0.20051357930526137, Validation Accuracy: 0.9316666666666666
Epoch 7/10, Loss: 0.19404546643917758, Validation Accuracy: 0.4416666666666666